# **Algoritmos Genéricos**

<p style="font-size: smaller; text-align: right;">João Dias & Rafael Rodrigues</p>

---

## **Objetivo**

Aplicar um Algoritmo Genético (AG) para encontrar o valor máximo da função 
𝑓(𝑥)=𝑥2−3𝑥+4f(x)=x2−3x+4 no intervalo 𝑋=[−10,+10].

---

## **Estrutura do Notebook**

1. **Introdução**  
   Explicação sobre os algoritmos genéticos e sua aplicação na otimização de funções.

2. **Representação e População Inicial**  
   Codificação dos indivíduos e geração da população inicial.

3. **Função Objetivo**  
   Implementação da função 𝑓(𝑥).


4. **Operadores Genéticos**  
   Implementação das operações de seleção, crossover e mutação.

5. **Execução do Algoritmo**  
   Evolução ao longo de várias gerações e visualização dos resultados.

6. **Conclusão**
Análise dos resultados e avaliação da eficácia do algoritmo.

---

## **Introdução**

Os Algoritmos Genéticos (AGs) são meta-heurísticas inspiradas na seleção natural e evolução biológica. Eles utilizam uma população de soluções candidatas que evoluem ao longo de gerações por meio de:

- **Seleção:** Escolha dos melhores indivíduos com base em sua aptidão.
- **Crossover:** Combinação de características entre indivíduos selecionados.
- **Mutação:** Introdução de pequenas variações para manter a diversidade.

Neste notebook, aplicaremos um AG para encontrar a solução que maximiza a função 𝑓(𝑥)no intervalo definido, utilizando:
- População inicial de *4 indivíduos*.
- Taxas de *crossover (70%)* e *mutação (1%)*.
- *Seleção por torneio* como método de escolha.
- Avaliação ao longo de *5 gerações*, podendo ser estendido para *20 gerações*.




---



## **Representação dos indivíduos e População inicial**

Para implementar o Algoritmo Genético (AG), precisamos codificar os valores de 𝑥 como*vetores bináriosVamos dividir essa etapa em três partes:

Codificação dos Indivíduos: Representação de valores de 
𝑥
x em formato binário.
Decodificação: Conversão do vetor binário de volta para valores reais no intervalo 
[
−
10
,
10
]
[−10,10].
Geração da População Inicial: Criação de uma população inicial de 4 indivíduos.